In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import warnings
import time
warnings.filterwarnings('ignore')

In [2]:
# ============================================================================
# CONFIGURATION
# ============================================================================
ERP_ID = 27857
USE_SAMPLING = False
SAMPLE_SIZE = 50000
USE_FEATURE_SELECTION = True
MAX_FEATURES = 30

np.random.seed(ERP_ID)

print("="*80)
print("RUSSIAN HOUSING PRICE PREDICTION - TASK 1")
print("="*80)
print(f"ERP ID: {ERP_ID}")
print(f"Configuration: Sampling={USE_SAMPLING}, Feature Selection={USE_FEATURE_SELECTION}")

RUSSIAN HOUSING PRICE PREDICTION - TASK 1
ERP ID: 27857
Configuration: Sampling=False, Feature Selection=True


In [4]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def stratified_sample(df, target_col='price_doc', sample_size=50000, random_state=None):
    """Stratified sampling preserving target distribution"""
    if len(df) <= sample_size:
        return df
    df['price_bin'] = pd.qcut(df[target_col], q=10, labels=False, duplicates='drop')
    sampled = df.groupby('price_bin', group_keys=False).apply(
        lambda x: x.sample(frac=sample_size/len(df), random_state=random_state)
    )
    return sampled.drop('price_bin', axis=1)

def create_features(df):
    """Feature engineering"""
    df = df.copy()
    if 'build_year' in df.columns:
        df['building_age'] = (2015 - df['build_year']).clip(lower=0)
    if 'full_sq' in df.columns and 'num_room' in df.columns:
        df['sqm_per_room'] = df['full_sq'] / (df['num_room'] + 1)
    if 'life_sq' in df.columns and 'full_sq' in df.columns:
        df['living_area_ratio'] = df['life_sq'] / (df['full_sq'] + 1)
    if 'kitch_sq' in df.columns and 'full_sq' in df.columns:
        df['kitchen_area_ratio'] = df['kitch_sq'] / (df['full_sq'] + 1)
    if 'floor' in df.columns and 'max_floor' in df.columns:
        df['floor_ratio'] = df['floor'] / (df['max_floor'] + 1)
        df['is_first_floor'] = (df['floor'] == 1).astype(int)
        df['is_last_floor'] = (df['floor'] == df['max_floor']).astype(int)
    return df

def quick_feature_selection(X, y, n_features=30, random_state=None):
    """Feature selection using multiple methods"""
    print(f"\n[Feature Selection] Selecting top {n_features} features...")
    numeric_X = X.select_dtypes(include=[np.number])
    
    if numeric_X.shape[1] <= n_features:
        return numeric_X.columns.tolist()
    
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(numeric_X)
    
    # Method 1: Correlation
    correlations = numeric_X.corrwith(y).abs()
    top_corr = correlations.nlargest(n_features).index.tolist()
    
    # Method 2: F-statistic
    selector = SelectKBest(score_func=f_regression, k=n_features)
    selector.fit(X_imputed, y)
    top_fstat = numeric_X.columns[selector.get_support()].tolist()
    
    # Method 3: Random Forest
    print("  Training Random Forest...")
    rf = RandomForestRegressor(n_estimators=50, max_depth=8, random_state=random_state, n_jobs=1)
    rf.fit(X_imputed, y)
    importances = pd.Series(rf.feature_importances_, index=numeric_X.columns)
    top_rf = importances.nlargest(n_features).index.tolist()
    
    # Consensus
    from collections import Counter
    all_features = top_corr + top_fstat + top_rf
    feature_votes = Counter(all_features)
    selected = [f for f, votes in feature_votes.most_common() if votes >= 2]
    
    if len(selected) < n_features:
        for f in top_rf:
            if f not in selected:
                selected.append(f)
            if len(selected) >= n_features:
                break
    
    print(f"  Selected {len(selected[:n_features])} features")
    return selected[:n_features]

In [5]:
# ============================================================================
# STEP 1: DATA LOADING & SAMPLING
# ============================================================================
print("\n" + "="*80)
print("STEP 1: DATA LOADING & SAMPLING")
print("="*80)

train_df = pd.read_csv('train.csv')
print(f"Original data: {train_df.shape}")

if USE_SAMPLING:
    train_df = stratified_sample(train_df, 'price_doc', SAMPLE_SIZE, ERP_ID)
    print(f"Sampled data: {train_df.shape}")


STEP 1: DATA LOADING & SAMPLING
Original data: (181507, 279)


In [6]:
# ============================================================================
# STEP 2: FEATURE ENGINEERING
# ============================================================================
print("\n" + "="*80)
print("STEP 2: FEATURE ENGINEERING")
print("="*80)

train_df = create_features(train_df)
X = train_df.drop('price_doc', axis=1)
y = train_df['price_doc']

# Remove non-predictive columns
cols_to_drop = ['id', 'timestamp']
X = X.drop([c for c in cols_to_drop if c in X.columns], axis=1)


STEP 2: FEATURE ENGINEERING
Features after engineering: 278


In [7]:
# ============================================================================
# STEP 3: TRAIN-VALIDATION SPLIT (70-30)
# ============================================================================
print("\n" + "="*80)
print("STEP 3: TRAIN-VALIDATION SPLIT (70-30)")
print("="*80)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=ERP_ID
)
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Random state: {ERP_ID}")


STEP 3: TRAIN-VALIDATION SPLIT (70-30)
Training set: (127054, 278)
Validation set: (54453, 278)
Random state: 27857


In [8]:
# ============================================================================
# STEP 4: FEATURE SELECTION
# ============================================================================
print("\n" + "="*80)
print("STEP 4: FEATURE SELECTION")
print("="*80)

if USE_FEATURE_SELECTION:
    selected_features = quick_feature_selection(X_train, y_train, MAX_FEATURES, ERP_ID)
    X_train_selected = X_train[selected_features]
    X_val_selected = X_val[selected_features]
    print(f"\n✓ Using {len(selected_features)} selected numeric features")
else:
    X_train_selected = X_train.select_dtypes(include=[np.number])
    X_val_selected = X_val.select_dtypes(include=[np.number])
    selected_features = X_train_selected.columns.tolist()


STEP 4: FEATURE SELECTION

[Feature Selection] Selecting top 30 features...
  Training Random Forest...
  Selected 30 features

✓ Using 30 selected features


In [9]:
print(selected_features)

['full_sq', 'leisure_count_1000', 'leisure_count_500', 'cafe_count_500_price_4000', 'trc_sqm_500', 'mosque_count_500', 'cafe_count_1000_price_high', 'cafe_count_500_price_high', 'cafe_count_1500_price_high', 'cafe_count_1000_price_1500', 'cafe_count_500_price_2500', 'culture_objects_top_25_raion', 'large_apartment', 'cafe_count_500_price_1000', 'leisure_count_1500', 'cafe_count_1000_price_4000', 'big_church_count_1000', 'office_sqm_1000', 'big_church_count_500', 'office_count_500', 'cafe_count_500_price_1500', 'cafe_count_1000_price_2500', 'cafe_count_500_na_price', 'cafe_count_1000_price_1000', 'cafe_count_1000', 'cafe_count_1500_price_2500', 'cafe_count_500', 'cafe_count_500_price_500', 'office_sqm_500', 'cafe_count_1000_price_500']


In [ ]:
# ============================================================================
# ADD CATEGORICAL FEATURES TO FEATURE SET
# ============================================================================
print("\n" + "="*80)
print("ENHANCING FEATURE SET WITH CATEGORICAL FEATURES")
print("="*80)

# Identify categorical features
categorical_features_all = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"\n✓ Found {len(categorical_features_all)} categorical features in dataset")

if len(categorical_features_all) > 0:
    # Show what we found
    print("  Examples:")
    for cat in categorical_features_all[:5]:
        n_unique = X_train[cat].nunique()
        print(f"    • {cat}: {n_unique} categories")
    
    # Filter: Keep only reasonable cardinality (2-50 unique values)
    categorical_selected = []
    for cat in categorical_features_all:
        n_unique = X_train[cat].nunique()
        if 2 <= n_unique <= 50:
            categorical_selected.append(cat)
    
    print(f"\n✓ Selected {len(categorical_selected)} categorical features (2-50 categories)")
else:
    categorical_selected = []

# Combine numeric + categorical
final_features = selected_features + categorical_selected
X_train_combined = X_train[final_features]
X_val_combined = X_val[final_features]

print(f"\n✓ FINAL FEATURE SET:")
print(f"  Numeric:      {len(selected_features)}")
print(f"  Categorical:  {len(categorical_selected)}")
print(f"  Total:        {len(final_features)}")

In [ ]:
print(selected_features)

In [10]:
# ============================================================================
# REQUIREMENT #4: PROPER PREPROCESSING PIPELINE
# ============================================================================
print("\n" + "="*80)
print("REQUIREMENT #4: PROPER PREPROCESSING PIPELINE")
print("="*80)

# NEW CODE - Identify feature types in COMBINED set
numeric_features = X_train_combined.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train_combined.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\n✓ Numeric features: {len(numeric_features)}")
print(f"✓ Categorical features: {len(categorical_features)}")

# Numeric transformer
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Median for numeric
    ('scaler', StandardScaler())                     # Standardize
])

# Categorical transformer - WITH MODE FOR MISSING VALUES
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # MODE for categorical
    ('onehot', OneHotEncoder(
        handle_unknown='ignore',
        sparse_output=False,
        max_categories=20
    ))
])

# Combine
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

print("\n✓ Preprocessing Pipeline Created:")
print("  [Numeric]     Median imputation → Standard scaling")
print("  [Categorical] MODE imputation → One-hot encoding")


REQUIREMENT #4: PROPER PREPROCESSING PIPELINE

✓ Numeric features: 30
✓ Categorical features: 0

✓ Preprocessing Pipeline Created:
  - Missing value imputation (median for numeric, constant for categorical)
  - One-hot encoding for categorical features
  - Standard scaling for numeric features


In [11]:
# ============================================================================
# MODEL TRAINING & EVALUATION
# ============================================================================
print("\n" + "="*80)
print("STEP 6: MODEL TRAINING & EVALUATION")
print("="*80)

results = {}

def evaluate_model(name, model, X_train, X_val, y_train, y_val):
    """Train and evaluate model"""
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    start_time = time.time()
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    inference_time = time.time() - start_time
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    
    results[name] = {
        'train_rmse': train_rmse,
        'val_rmse': val_rmse,
        'train_r2': train_r2,
        'val_r2': val_r2,
        'train_time': train_time,
        'inference_time': inference_time,
        'overfitting_gap': train_rmse - val_rmse
    }
    
    print(f"\n{name}")
    print(f"  Val RMSE: {val_rmse:.4f} | Train RMSE: {train_rmse:.4f}")
    print(f"  Val R²: {val_r2:.4f} | Train R²: {train_r2:.4f}")
    print(f"  Training: {train_time:.2f}s | Inference: {inference_time:.4f}s")
    
    return model


STEP 6: MODEL TRAINING & EVALUATION


In [12]:
# ============================================================================
# REQUIREMENT #3 & #6: Baseline Linear Regression
# ============================================================================
print("\n--- 6.1: BASELINE LINEAR REGRESSION ---")

baseline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

baseline_model = evaluate_model(
    'Baseline Linear', baseline, 
    X_train_combined, X_val_combined, y_train, y_val
)


--- 6.1: BASELINE LINEAR REGRESSION ---

Baseline Linear
  Val RMSE: 13.2984 | Train RMSE: 13.2966
  Val R²: 0.6226 | Train R²: 0.6169
  Training: 3.68s | Inference: 0.4208s


In [13]:
# ============================================================================
# REQUIREMENT #5: POLYNOMIAL REGRESSION WITH GRIDSEARCHCV
# ============================================================================
print("\n--- 6.2: POLYNOMIAL REGRESSION (WITH GRIDSEARCHCV) ---")
print("\n⚠️  NOTE: The assignment requires using GridSearchCV to tune polynomial")
print("         degree and interaction_only parameter, not manual training.")

poly_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('regressor', LinearRegression())
])

# Parameter grid for polynomial tuning
param_grid_poly = {
    'poly__degree': [2, 3],  # Degree 2 and 3 as required
    'poly__interaction_only': [False, True]  # With and without interactions
}

print("\nSearching for best polynomial configuration...")
print(f"Testing: {param_grid_poly}")

poly_search = GridSearchCV(
    poly_pipeline, 
    param_grid_poly, 
    cv=3,  # 3-fold CV for speed
    scoring='neg_root_mean_squared_error',
    n_jobs=1,
    verbose=1
)

poly_search.fit(X_train_selected, y_train)

print(f"\n✓ Best polynomial configuration found:")
print(f"  - Degree: {poly_search.best_params_['poly__degree']}")
print(f"  - Interaction only: {poly_search.best_params_['poly__interaction_only']}")
print(f"  - CV Score: {-poly_search.best_score_:.4f} RMSE")

# Evaluate best polynomial model
best_poly = evaluate_model(
    'Polynomial (Best)', poly_search.best_estimator_,
    X_train_combined, X_val_combined, y_train, y_val
)

# Show all polynomial configurations tested
print("\n✓ All polynomial configurations tested:")
cv_results = pd.DataFrame(poly_search.cv_results_)
for idx, row in cv_results.iterrows():
    print(f"  {row['params']}: CV RMSE = {-row['mean_test_score']:.4f}")


--- 6.2: POLYNOMIAL REGRESSION (WITH GRIDSEARCHCV) ---

⚠️  NOTE: The assignment requires using GridSearchCV to tune polynomial
         degree and interaction_only parameter, not manual training.

Searching for best polynomial configuration...
Testing: {'poly__degree': [2, 3], 'poly__interaction_only': [False, True]}
Fitting 3 folds for each of 4 candidates, totalling 12 fits

✓ Best polynomial configuration found:
  - Degree: 2
  - Interaction only: True
  - CV Score: 13.1697 RMSE

Polynomial (Best)
  Val RMSE: 13.0829 | Train RMSE: 12.8238
  Val R²: 0.6348 | Train R²: 0.6437
  Training: 13.41s | Inference: 2.8239s

✓ All polynomial configurations tested:
  {'poly__degree': 2, 'poly__interaction_only': False}: CV RMSE = 13.1798
  {'poly__degree': 2, 'poly__interaction_only': True}: CV RMSE = 13.1697
  {'poly__degree': 3, 'poly__interaction_only': False}: CV RMSE = nan
  {'poly__degree': 3, 'poly__interaction_only': True}: CV RMSE = nan


In [15]:
# ============================================================================
# REQUIREMENT #5: RIDGE REGRESSION WITH GRIDSEARCHCV
# ============================================================================
print("\n--- 6.3: RIDGE REGRESSION (L2) WITH GRIDSEARCHCV ---")

ridge_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

param_grid_ridge = {
    'regressor__alpha': [0.01, 0.1, 1, 10, 100, 1000]
}

print(f"\nTuning Ridge regularization strength: {param_grid_ridge['regressor__alpha']}")

ridge_search = GridSearchCV(
    ridge_pipeline, param_grid_ridge, cv=3,
    scoring='neg_root_mean_squared_error', n_jobs=1
)

ridge_search.fit(X_train_selected, y_train)

print(f"\n✓ Best Ridge alpha: {ridge_search.best_params_['regressor__alpha']}")
print(f"  CV Score: {-ridge_search.best_score_:.4f} RMSE")

best_ridge = evaluate_model(
    'Ridge (Best)', ridge_search.best_estimator_,
    X_train_combined, X_val_combined, y_train, y_val
)


--- 6.3: RIDGE REGRESSION (L2) WITH GRIDSEARCHCV ---

Tuning Ridge regularization strength: [0.01, 0.1, 1, 10, 100, 1000]

✓ Best Ridge alpha: 1000
  CV Score: 13.3195 RMSE

Ridge (Best)
  Val RMSE: 13.2987 | Train RMSE: 13.2968
  Val R²: 0.6226 | Train R²: 0.6169
  Training: 1.70s | Inference: 0.3668s


In [17]:
# ============================================================================
# REQUIREMENT #5: LASSO REGRESSION WITH GRIDSEARCHCV
# ============================================================================
print("\n--- 6.4: LASSO REGRESSION (L1) WITH GRIDSEARCHCV ---")

lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso(max_iter=10000))
])

param_grid_lasso = {
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

print(f"\nTuning Lasso regularization strength: {param_grid_lasso['regressor__alpha']}")

lasso_search = GridSearchCV(
    lasso_pipeline, param_grid_lasso, cv=3,
    scoring='neg_root_mean_squared_error', n_jobs=1
)

lasso_search.fit(X_train_selected, y_train)

print(f"\n✓ Best Lasso alpha: {lasso_search.best_params_['regressor__alpha']}")
print(f"  CV Score: {-lasso_search.best_score_:.4f} RMSE")

best_lasso = evaluate_model(
    'Lasso (Best)', lasso_search.best_estimator_,
    X_train_combined, X_val_combined, y_train, y_val
)


--- 6.4: LASSO REGRESSION (L1) WITH GRIDSEARCHCV ---

Tuning Lasso regularization strength: [0.001, 0.01, 0.1, 1, 10, 100]

✓ Best Lasso alpha: 0.1
  CV Score: 13.3180 RMSE

Lasso (Best)
  Val RMSE: 13.2978 | Train RMSE: 13.2981
  Val R²: 0.6227 | Train R²: 0.6169
  Training: 4.98s | Inference: 0.3828s


In [18]:
# ============================================================================
# REQUIREMENT #5: ELASTIC NET WITH GRIDSEARCHCV
# ============================================================================
print("\n--- 6.5: ELASTIC NET WITH GRIDSEARCHCV ---")

elastic_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ElasticNet(max_iter=10000))
])

param_grid_elastic = {
    'regressor__alpha': [0.01, 0.1, 1, 10],
    'regressor__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

print(f"\nTuning Elastic Net hyperparameters:")
print(f"  Alpha: {param_grid_elastic['regressor__alpha']}")
print(f"  L1 ratio: {param_grid_elastic['regressor__l1_ratio']}")

elastic_search = GridSearchCV(
    elastic_pipeline, param_grid_elastic, cv=3,
    scoring='neg_root_mean_squared_error', n_jobs=-1
)

elastic_search.fit(X_train_selected, y_train)

print(f"\n✓ Best Elastic Net parameters:")
print(f"  Alpha: {elastic_search.best_params_['regressor__alpha']}")
print(f"  L1 ratio: {elastic_search.best_params_['regressor__l1_ratio']}")
print(f"  CV Score: {-elastic_search.best_score_:.4f} RMSE")

best_elastic = evaluate_model(
    'Elastic Net (Best)', elastic_search.best_estimator_,
    X_train_combined, X_val_combined, y_train, y_val
)


--- 6.5: ELASTIC NET WITH GRIDSEARCHCV ---

Tuning Elastic Net hyperparameters:
  Alpha: [0.01, 0.1, 1, 10]
  L1 ratio: [0.1, 0.3, 0.5, 0.7, 0.9]

✓ Best Elastic Net parameters:
  Alpha: 0.1
  L1 ratio: 0.9
  CV Score: 13.3177 RMSE

Elastic Net (Best)
  Val RMSE: 13.2983 | Train RMSE: 13.2983
  Val R²: 0.6226 | Train R²: 0.6168
  Training: 4.94s | Inference: 0.3334s


In [19]:
# ============================================================================
# REQUIREMENT #7: GRADIENT BOOSTING (LightGBM)
# ============================================================================
print("\n" + "="*80)
print("REQUIREMENT #7: GRADIENT BOOSTING MODEL (LightGBM)")
print("="*80)

# Prepare data for LightGBM
X_train_lgb = X_train_combined.select_dtypes(include=[np.number])
X_val_lgb = X_val_combined.select_dtypes(include=[np.number])

imputer = SimpleImputer(strategy='median')
X_train_lgb_imputed = pd.DataFrame(
    imputer.fit_transform(X_train_lgb),
    columns=X_train_lgb.columns
)
X_val_lgb_imputed = pd.DataFrame(
    imputer.transform(X_val_lgb),
    columns=X_val_lgb.columns
)

print("\nTraining LightGBM...")

start_time = time.time()

lgb_model = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=ERP_ID,
    verbose=-1
)

lgb_model.fit(
    X_train_lgb_imputed, y_train,
    eval_set=[(X_val_lgb_imputed, y_val)],
    callbacks=[lgb.early_stopping(50, verbose=False)]
)

lgb_train_time = time.time() - start_time

# Evaluate
start_time = time.time()
y_train_pred_lgb = lgb_model.predict(X_train_lgb_imputed)
y_val_pred_lgb = lgb_model.predict(X_val_lgb_imputed)
lgb_inference_time = time.time() - start_time

results['LightGBM'] = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_train_pred_lgb)),
    'val_rmse': np.sqrt(mean_squared_error(y_val, y_val_pred_lgb)),
    'train_r2': r2_score(y_train, y_train_pred_lgb),
    'val_r2': r2_score(y_val, y_val_pred_lgb),
    'train_time': lgb_train_time,
    'inference_time': lgb_inference_time,
    'overfitting_gap': np.sqrt(mean_squared_error(y_train, y_train_pred_lgb)) - 
                       np.sqrt(mean_squared_error(y_val, y_val_pred_lgb))
}

print(f"\nLightGBM")
print(f"  Val RMSE: {results['LightGBM']['val_rmse']:.4f} | "
      f"Train RMSE: {results['LightGBM']['train_rmse']:.4f}")
print(f"  Val R²: {results['LightGBM']['val_r2']:.4f} | "
      f"Train R²: {results['LightGBM']['train_r2']:.4f}")
print(f"  Training: {lgb_train_time:.2f}s | Inference: {lgb_inference_time:.4f}s")


REQUIREMENT #7: GRADIENT BOOSTING MODEL (LightGBM)

Training LightGBM...


  File "C:\Users\SHEIKHANI LAPTOP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 247, in _count_physical_cores
    cpu_count_physical = _count_physical_cores_win32()
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SHEIKHANI LAPTOP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 299, in _count_physical_cores_win32
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py


LightGBM
  Val RMSE: 12.7639 | Train RMSE: 12.3146
  Val R²: 0.6524 | Train R²: 0.6714
  Training: 6.73s | Inference: 0.9828s


In [20]:
# ============================================================================
# REQUIREMENT #6: COMPREHENSIVE RESULTS SUMMARY
# ============================================================================
print("\n" + "="*80)
print("REQUIREMENT #6: COMPREHENSIVE RESULTS SUMMARY")
print("="*80)

results_df = pd.DataFrame(results).T

print("\n✓ VALIDATION RMSE FOR ALL MODELS:")
print("="*50)
for model_name in ['Baseline Linear', 'Polynomial (Best)', 'Ridge (Best)', 
                    'Lasso (Best)', 'Elastic Net (Best)']:
    if model_name in results_df.index:
        print(f"{model_name:25s}: {results_df.loc[model_name, 'val_rmse']:.4f} million RUB")

print("\n✓ BEST LINEAR MODEL IDENTIFICATION:")
print("="*50)
linear_models = results_df.drop('LightGBM', errors='ignore')
best_linear_name = linear_models['val_rmse'].idxmin()
best_linear_rmse = linear_models['val_rmse'].min()
print(f"Best Linear Model: {best_linear_name}")
print(f"Validation RMSE: {best_linear_rmse:.4f} million RUB")

print("\n✓ COMPARISON WITH GRADIENT BOOSTING:")
print("="*50)
lgb_rmse = results_df.loc['LightGBM', 'val_rmse']
improvement = ((best_linear_rmse - lgb_rmse) / best_linear_rmse * 100)
print(f"LightGBM Validation RMSE: {lgb_rmse:.4f} million RUB")
print(f"Improvement over best linear: {improvement:.2f}%")

print("\n✓ DETAILED PERFORMANCE COMPARISON:")
print("="*50)
comparison_df = results_df[['val_rmse', 'train_rmse', 'val_r2', 'train_time', 'inference_time']]
comparison_df = comparison_df.sort_values('val_rmse')
print(comparison_df.to_string())

print("\n✓ OVERFITTING ANALYSIS:")
print("="*50)
for model_name, gap in results_df['overfitting_gap'].items():
    status = "✓ Good" if abs(gap) < 2 else "⚠️  Overfitting" if gap < -2 else "⚠️  Underfitting"
    print(f"{model_name:25s}: Gap = {gap:+.4f} {status}")

print("\n" + "="*80)
print("TASK 1 COMPLETED SUCCESSFULLY")
print("="*80)
print("\n✓ All requirements met:")
print("  [✓] Requirement 3: All model types implemented")
print("  [✓] Requirement 4: Complete preprocessing pipeline")
print("  [✓] Requirement 5: GridSearchCV for all tunable models")
print("  [✓] Requirement 6: Comprehensive RMSE reporting")
print("  [✓] Requirement 7: Gradient boosting trained and compared")

# Save results for report
results_df.to_csv('model_results.csv')
print("\n✓ Results saved to 'model_results.csv'")


REQUIREMENT #6: COMPREHENSIVE RESULTS SUMMARY

✓ VALIDATION RMSE FOR ALL MODELS:
Baseline Linear          : 13.2984 million RUB
Polynomial (Best)        : 13.0829 million RUB
Ridge (Best)             : 13.2987 million RUB
Lasso (Best)             : 13.2978 million RUB
Elastic Net (Best)       : 13.2983 million RUB

✓ BEST LINEAR MODEL IDENTIFICATION:
Best Linear Model: Polynomial (Best)
Validation RMSE: 13.0829 million RUB

✓ COMPARISON WITH GRADIENT BOOSTING:
LightGBM Validation RMSE: 12.7639 million RUB
Improvement over best linear: 2.44%

✓ DETAILED PERFORMANCE COMPARISON:
                     val_rmse  train_rmse    val_r2  train_time  inference_time
LightGBM            12.763921   12.314561  0.652351    6.727606        0.982783
Polynomial (Best)   13.082928   12.823797  0.634756   13.414179        2.823859
Lasso (Best)        13.297763   13.298104  0.622663    4.983117        0.382789
Elastic Net (Best)  13.298348   13.298342  0.622629    4.938836        0.333422
Baseline Linear 